In [ ]:
# Imports and environment checks
import os
import sys
import torch
from pathlib import Path
print('Python:', sys.version.split()[0])
print('Torch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('CUDA device:', torch.cuda.get_device_name(0))

# check data dirs
base = Path('..') if Path('..').exists() else Path('.')
data_root = Path('..') / 'data' if (Path('..') / 'data').exists() else Path('data')
print('Data root:', data_root)

In [ ]:
# This cell only defines the training call. Run with care (GPU required for reasonable speed).
from ultralytics import YOLO

# Choose nano model (yolov8s.pt for small model)
model = YOLO('yolov8n.pt')

train_args = dict(
    data='data/data.yaml',
    epochs=50,
    imgsz=640,
    batch=8,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=4,
    name='yolov8_experiment',
    pretrained=True,
    save=True
)

print('Training config:', train_args)
model.train(**train_args)

In [ ]:
from ultralytics import YOLO
from IPython.display import display
import pandas as pd
import os
from pathlib import Path
import random
import json

# Load model from given path (weights path or pretrained)
m = YOLO('../runs/detect/yolov8_experiment/weights/best.pt')

# Utility: find a random image containing a given class name
# Strategy: search labels in ../data/test/labels for lines starting with that class id,
# map to image filename, then pick one at random.

def load_class_map(model: YOLO):
    if hasattr(model, 'names') and isinstance(model.names, (dict, list)):
        if isinstance(model.names, dict):
            return {int(k): v for k, v in model.names.items()}
        else:
            return {i: name for i, name in enumerate(model.names)}
    return {}

CLASS_MAP = load_class_map(m)
INV_CLASS_MAP = {v.lower(): k for k, v in CLASS_MAP.items()}  # name -> id

labels_dir = Path('../data/test/labels')
images_dir = Path('../data/test/images')

# Helper to resolve candidate images for a given class id

def images_for_class(class_id: int):
    candidates = []
    if not labels_dir.exists():
        return candidates
    for lbl in labels_dir.glob('*.txt'):
        try:
            with open(lbl, 'r') as f:
                lines = [ln.strip() for ln in f if ln.strip()]
            # any line with the right class id as first token?
            for ln in lines:
                parts = ln.split()
                if len(parts) >= 1 and parts[0].isdigit() and int(parts[0]) == class_id:
                    # find image file with same stem
                    for ext in ('.jpg', '.jpeg', '.png'):
                        img_path = images_dir / f"{lbl.stem}{ext}"
                        if img_path.exists():
                            candidates.append(img_path)
                            break
                    break
        except Exception:
            pass
    return candidates

# User input: provide insect/class name (case-insensitive). If empty, fallback to any test image.
INSECT_NAME = 'Caterpillars'  # e.g., 'Ants', 'Beetles', 'Caterpillars'

if INSECT_NAME:
    cls_id = INV_CLASS_MAP.get(INSECT_NAME.lower())
    if cls_id is None:
        raise ValueError(f"Unknown class name '{INSECT_NAME}'. Known: {sorted(set(CLASS_MAP.values()))}")
    pool = images_for_class(cls_id)
    if not pool:
        raise FileNotFoundError(f"No images found in test set containing class '{INSECT_NAME}'.")
    sample_img = random.choice(pool)
else:
    # Fallback: pick any image from test/images
    image_dir = images_dir
    candidates = []
    for ext in ('.jpg', '.jpeg', '.png'):
        candidates.extend(list(image_dir.glob(f'*{ext}')))
    if not candidates:
        raise FileNotFoundError('No images found in test/images directory.')
    sample_img = random.choice(candidates)

print('Running prediction on:', sample_img)
results = m.predict(source=str(sample_img), save=False, imgsz=640, conf=0.25)

# results is a list; take first item
res = results[0]
boxes = res.boxes

# Extract detection data
if boxes is not None and len(boxes) > 0:
    xyxy = boxes.xyxy.cpu().numpy()
    conf = boxes.conf.cpu().numpy()
    cls = boxes.cls.cpu().numpy()

    # Build a DataFrame for readability
    df = pd.DataFrame(xyxy, columns=['x1','y1','x2','y2'])
    df['confidence'] = conf
    df['class_id'] = cls.astype(int)
    df['class_name'] = df['class_id'].map(CLASS_MAP)

    # Sort by confidence descending
    df = df.sort_values('confidence', ascending=False).reset_index(drop=True)
    display(df.head(10))
else:
    print('No detections found.')

# Annotated image rendering and save
annotated = res.plot()
import cv2
annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
plt.imshow(annotated_rgb)
plt.axis('off')
plt.title('Detections')
plt.show()

# Ensure output dir exists
out_dir = Path(f'../results/yolov8_predictions/{INSECT_NAME}')
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / f'annotated_{sample_img.name}'
cv2.imwrite(str(out_path), annotated)
print('Annotated image saved to:', out_path)
print('Inference summary: total detections =', 0 if boxes is None else len(boxes))